In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import holoviews as hv
hv.notebook_extension()

df = pd.read_hdf('data/conductance.hdf')
df['B'] = np.sqrt(df.B_x**2 +  df.B_y**2 +  df.B_z**2)
df['theta'] = np.rad2deg(np.arccos(df.B_z / df.B))
df['phi'] = np.rad2deg(np.arctan(df.B_y / df.B_x))
df['direction'] = np.argmax(np.array([df.B_x, df.B_y, df.B_z]), axis=0)
print(len(df))

In [ ]:
params = ['alpha', 'g', 'orbital', 'V_barrier', 'mu', 'direction']
gb = df.groupby(params)

hm = {}
hm2 = {}
for i, gr in gb:
    direction = i[-1]
    B = 'B_{}'.format('xyz'[direction])

    opts = dict(kdims=['V_bias', 'B_x'], vdims=['G'],
                bounds=(gr['V_bias'].min(), gr[B].min(), gr['V_bias'].max(), gr[B].max()))

    hm[i] = hv.Image(np.flipud(gr.pivot_table('G_Andreev', B, 'V_bias')), **opts)
    hm2[i] = hv.Image(np.flipud(gr.pivot_table('G_01', B, 'V_bias')), **opts)

holomap = hv.HoloMap(hm, kdims=params, label='G_Andreev')
holomap2 = hv.HoloMap(hm2, kdims=params, label='G_01')

In [ ]:
%%opts Image {+axiswise} [colorbar=True aspect='square']
%%output size=200
hv.util.Dynamic(holomap) + hv.util.Dynamic(holomap2)